In [ ]:
pip install ultralytics

In [ ]:
import os

# Create folders to store the images
frame_box_folder = "/content/drive/MyDrive/SKY/Data/car_frame_box"
frame_img_folder = "/content/drive/MyDrive/SKY/Data/car_frame_img"

# Check if the folders exist, if not, create them
if not os.path.exists(frame_box_folder):
    os.makedirs(frame_box_folder)

if not os.path.exists(frame_img_folder):
    os.makedirs(frame_img_folder)

In [ ]:
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import random
import json
import cv2

# opening the file in read mode
my_file = open("data/coco.txt", "r")
# reading the file
data = my_file.read()
# replacing end splitting the text | when newline ('\n') is seen.
class_list = data.split("\n")
# print(class_list)
my_file.close()

# Generate random colors for class list
detection_colors = []
for i in range(len(class_list)):
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    detection_colors.append((b, g, r))

# load a pretrained YOLOv8n model
model = YOLO("data/yolov8n.pt", "v8")

source = "Video/lpCamera_1h.avi"
models = model.track(source = source, classes = [2,7], conf = 0.25, tracker = "data/botsort.yaml", agnostic_nms = True, stream = True)
vdo = cv2.VideoCapture(source)

if not vdo.isOpened():
    print("Cannot open Video")
    exit()

# Get the size of the video
width = int(vdo.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vdo.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = vdo.get(cv2.CAP_PROP_FPS)
frame_count = int(vdo.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Video Size: {width}x{height}")
print(f"Frame Rate: {fps} frames per second")
print(f"Total Frames: {frame_count}")

# Initialize polygon box coordinates as a list of points forming the scalene rectangle(x, y)
poly_box = [(1100, 450), (2050, 450), (1950, 650), (900, 650)]

# Store the last value
track_last = 0
class_last = 0

# Store the previous last value
prev_track_last = 0

# Initialize a frame counter
counter = 0

for frame in models:
    # if frame is read correctly ret is True
    if not frame:
        print("Can't receive frame (stream end). Exiting")
        continue

    # Create a copy of the frame to save with additional boxes
    frame_box = frame.orig_img
    # Create a copy of the frame to save without additional boxes
    frame_img = frame_box.copy()
    
    # Convert tensor array to numpy
    detect_params = frame[0].cpu().numpy()
    # print("--------------------------------------------------------------------------------------------------")
    # print(frame[0].boxes)

    if len(detect_params) != 0:
        # Flag to indicate if any car with confidence greater than required.
        car_detected = False
        # Variable to store the car's box midpoint
        car_midpoint = None
        bb = None
        # Variables to store the highest confidence box
        max_conf_box = None
        max_conf = -1
        for box in frame[0].boxes:
            if box.cls  is not None and box.id is not None:
                clsID = box.cls.cpu().numpy()[0]
                conf = box.conf.cpu().numpy()[0]
                trackID = box.id.cpu().numpy()[0]
                # print("Box: ", box[0])
                # print("Cls: ", clsID)
                # print("Conf: ", conf)
                # print("ID: ", trackID)

                # more than 50% confidence
                if conf > 0.50 and conf > max_conf:
                    car_detected = True
                    max_conf = conf
                    max_conf_box = box
            
        if max_conf_box is not None:
            clsID = max_conf_box.cls.cpu().numpy()[0]
            conf = max_conf_box.conf.cpu().numpy()[0]
            trackID = max_conf_box.id.cpu().numpy()[0]
            bb = max_conf_box.xyxy.cpu().cpu().numpy()[0]
            # Class that the track saw last time
            class_last = class_list[int(clsID)]
            track_last = int(trackID)
            # print("###############################################################################################")        
            # print("Class last: ", class_last)
            # print("Track last: ", track_last)
            
        if bb is not None:
            cv2.rectangle(
                frame_box,
                (int(bb[0]), int(bb[1])),
                (int(bb[2]), int(bb[3])),
                color = detection_colors[int(clsID)],
                thickness = 3
            )

            # Display class name and confidence
            font = cv2.FONT_HERSHEY_COMPLEX
            cv2.putText(
                frame_box,
                class_list[int(clsID)] + " " + str(round(conf, 3)) + "%",
                (int(bb[0] + 10), int(bb[3]) - 10),
                font,
                fontScale = 1,
                color = (255, 255, 255),
                thickness = 2
            )
            
            # Display class id
            cv2.putText(
                frame_box,
                "ID: " + str(track_last),
                (int(bb[0] + 10), int(bb[3]) + 30),
                font,
                fontScale = 1,
                color = (255, 255, 255),
                thickness = 2
            )
                    
            # Calculate and store the car's box midpoint
            car_midpoint = ((bb[0] + bb[2]) // 2, (bb[1] + bb[3]) // 2)
            
            # Check if the car's midpoint is inside the inner box
            point_inner_box = cv2.pointPolygonTest(np.array(poly_box), car_midpoint, False)
            
            # Display the resulting frame only if box midpoint inside the inner box
            if car_detected and car_midpoint is not None and point_inner_box >= 0:
                # print("bb", bb)
                # print("Car Midpoint:", car_midpoint)
                if  track_last > prev_track_last:
                    # Create inner box (scalene rectangle)
                    cv2.polylines(
                        frame_box,
                        [np.array(poly_box)],
                        isClosed = True,
                        color = (0, 255, 0),
                        thickness = 2
                    )
                    
                    # Draw a center point inside the bounding box
                    cv2.circle(frame_box, (int(car_midpoint[0]), int(car_midpoint[1])), 5, (0, 0, 255), -1)

                    # Display the image using matplotlib
                    plt.imshow(cv2.cvtColor(frame_box, cv2.COLOR_BGR2RGB))
                    plt.axis('off')  # Turn off axis ticks and labels
                    plt.show()

                    # Save the image every 1 frames
                    if counter % 1 == 0:
                        cv2.imwrite(os.path.join(frame_box_folder, f"image-{counter+1}.jpg"), frame_box)
                        cv2.imwrite(os.path.join(frame_img_folder, f"image-{counter+1}.jpg"), frame_img)
                    # Increment the frame counter
                    counter += 1
                    
                    prev_track_last = track_last

# When everything done, release the capture
vdo.release()
cv2.destroyAllWindows()

In [ ]:
import os

# Folders for storing the images
frame_box_folder = "data/car_frame_box"
frame_img_folder = "data/car_frame_img"

# Function to delete all image files from the specified folder
def delete_image_files(folder):
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):  # Check if the file is an image (JPEG format)
            file_path = os.path.join(folder, filename)
            os.remove(file_path)

# Check if the folders exist, if not, create them
if not os.path.exists(frame_box_folder):
    os.makedirs(frame_box_folder)

if not os.path.exists(frame_img_folder):
    os.makedirs(frame_img_folder)

# Delete image files from the folders
delete_image_files(frame_box_folder)
delete_image_files(frame_img_folder)